In [3]:
# https://docs.astral.sh/uv/getting-started/installation/
!uv add google-api-python-client pandas python-dotenv pendulum

            ▄▀▄       ▄      ▄                
            ▀▀▀▀  ▄ ▄▀▀▀▄▀▀▄▀▀▀               
            ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄▄             
             ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀            
              ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄          
             ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄          
     ▄▄▄    ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄        
  ▄▄▀▀▀▀▀▀▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀         
▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄        
 ▀▀▀▀▀▀▀▀▀▀ ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄▄   
   ▀▀▀▀▀▀   ▄▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀  
            ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▄ 
              ▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀▀
             ▄▀▀▀▀▀▀▀▀▀▀▀ ▀▀▀▀▀▀▀   ▀▀▀▀      
              ▀▀▀▀▀      ▄▀▀▀▀▀▀▀             
                         ▀▀▀▀▀▀▀              
                           ▀▀▀▀               

Resolved 128 packages in 0.48ms
Audited 124 packages in 0.98ms


In [4]:
import os
from dotenv import load_dotenv
from googleapiclient.discovery import build
import pandas as pd
import time
import pendulum

load_dotenv()

API_KEY = os.getenv("YT_DATA_API")
youtube = build(serviceName="youtube", version="v3", developerKey=API_KEY)

data_path='../data'

In [5]:
def get_videos(
    query: str,
    last_discovery,
    max_results=10,
    pages=1,
):
    video_data = []
    page_token = None

    for page in range(pages):
        print(f"finding {query} on page {page + 1}")
        req = youtube.search().list(
            q=query,
            part="snippet",
            type="video",
            maxResults=max_results,
            order="relevance",
            pageToken=page_token,
            publishedAfter=last_discovery.to_rfc3339_string(),  # https://developers.google.com/youtube/v3/docs/search/list#publishedAfter,
        )

        res = req.execute()

        for item in res.get("items", []):
            video_data.append(
                {
                    "video_id": item["id"]["videoId"],
                    "title": item["snippet"]["title"],
                    "description": item["snippet"]["description"],
                    "channel_id": item["snippet"]["channelId"],
                    "channel_title": item["snippet"]["channelTitle"],
                    "publish_date": item["snippet"]["publishedAt"],
                    "search_query": query,
                    "discovered_at": pendulum.now(),
                }
            )

        page_token = res.get("nextPageToken")
        if not page_token:
            break
        time.sleep(0.5)

    return video_data

In [6]:
topics = [
    "Zenless Zone Zero guide",
    "Zenless Zone Zero character showcase",
    "Zenless Zone Zero deadly assualt",
    "Zenless Zone Zero shiyu defence",
    "Zenless Zone Zero tier list",
    "Zenless Zone Zero news",
    "Zenless Zone Zero gameplay",
    "Zenless Zone Zero cosplay",
]
agents=pd.read_csv(f'{data_path}/dim_agent.csv')['name']
for agent in agents:
    topics.append(f'Zenless Zone Zero {agent}')

# comment for production
# topics=topics[:1]

In [7]:
videos = []
last_discovery=pendulum.datetime(2024,1,1)
# released 4 July 2024, but beta is a thing
for query in topics:
    videos.extend(get_videos(query, last_discovery, max_results=50, pages=1))

df = pd.DataFrame(videos)
df=df.drop_duplicates(subset=['video_id'])

finding Zenless Zone Zero guide on page 1
finding Zenless Zone Zero character showcase on page 1
finding Zenless Zone Zero deadly assualt on page 1
finding Zenless Zone Zero shiyu defence on page 1
finding Zenless Zone Zero tier list on page 1
finding Zenless Zone Zero news on page 1
finding Zenless Zone Zero gameplay on page 1
finding Zenless Zone Zero cosplay on page 1
finding Zenless Zone Zero Alexandrina Sebastiane on page 1
finding Zenless Zone Zero Alice Thymefield on page 1
finding Zenless Zone Zero Anby Demara on page 1
finding Zenless Zone Zero Anton Ivanov on page 1
finding Zenless Zone Zero Asaba Harumasa on page 1
finding Zenless Zone Zero Astra Yao on page 1
finding Zenless Zone Zero Banyue on page 1
finding Zenless Zone Zero Ben Bigger on page 1
finding Zenless Zone Zero Billy Kid on page 1
finding Zenless Zone Zero Burnice White on page 1
finding Zenless Zone Zero Caesar King on page 1
finding Zenless Zone Zero Corin Wickes on page 1
finding Zenless Zone Zero Dialyn on p

,video_id,title,description,channel_id,channel_title,publish_date,search_query,discovered_at
0,kR626gjg0MQ,Zenless Zone Zero Beginner&#39;s Guide - 20 Ti...,Zenless Zone Zero is here! Here are 20 tips an...,UCHMnsOPDgrjWvjHXZp-xvqg,IWinToLose Gaming,2024-07-04T21:50:14Z,Zenless Zone Zero guide,2026-01-22 07:38:00.695385+05:30
1,poJkFK3_M_0,How to get 180 FREE pulls in Zenless Zone Zero...,,UCKy1dAqELo0zrOtPkf0eTMw,IGN,2024-07-06T00:19:44Z,Zenless Zone Zero guide,2026-01-22 07:38:00.695404+05:30
2,6_znQMRcIxg,"If I Started Zenless Zone Zero In 2025, Here&#...",This beginner and new player friendly guide is...,UCSvjKqgK2kY4W0i2T5sTZBg,Rhinne,2025-08-01T16:16:54Z,Zenless Zone Zero guide,2026-01-22 07:38:00.695408+05:30
3,WSSWhnX0hgA,Zenless Zone Zero Combat Guide,"From basic controls and systems, to Bangboos a...",UCbu2SsF-Or3Rsn3NxqODImw,GameSpot,2024-07-04T02:00:19Z,Zenless Zone Zero guide,2026-01-22 07:38:00.695411+05:30
4,vD2hbBykyVM,A COMPLETE Guide to Combat Basics in Zenless Z...,"In this complete guide, I go through all the c...",UCDbRoMWfEYzvZ9RlKeSH1Yw,Guoba Uncertified,2024-07-08T14:45:36Z,Zenless Zone Zero guide,2026-01-22 07:38:00.695414+05:30
...,...,...,...,...,...,...,...,...
2667,De5XTvJzhos,🚨 Noticias Zenless Zone Zero — Así es SUNNA y ...,U7BUY Zenless Zone Zero — ¡Ahorra al máximo en...,UC1ySZZmGC81xg-aQJ7yV7Yw,Locbry,2026-01-02T08:04:35Z,Zenless Zone Zero Sunna,2026-01-22 07:39:13.602159+05:30
2670,4V-NnA7wZOY,ZZZ 2.6 Update: Aria &amp; Sunna Mega Niche?! ...,zzz #zenlesszonezero #sunna #zzzsunna #zenless...,UC3AfxChN7R81Hxa36hNJZWw,Razor_ct,2026-01-12T14:30:41Z,Zenless Zone Zero Sunna,2026-01-22 07:39:13.602164+05:30
2672,oGZsb6YBHF4,ZZZ 2.6v1 – M0 Sunna + Ye Shunguang + Zhao | D...,ZZZ 2.6v1 – M0 Sunna + Ye Shunguang + Zhao | D...,UC9L5puw-XKVnFYqkMcCjokQ,Yixuan exe,2026-01-07T12:39:56Z,Zenless Zone Zero Sunna,2026-01-22 07:39:13.602174+05:30
2673,VMJm0wWXHcI,ZZZ 2.6 - 3.2 ALL NEWEST BANNERS ROADMAP + RER...,ZZZ 2.6 - 3.2 ALL NEWEST BANNERS ROADMAP + RER...,UCHrSjTSYC7PJcA95Baf8CMA,Mista ZZZ,2026-01-07T12:01:04Z,Zenless Zone Zero Sunna,2026-01-22 07:39:13.602176+05:30


In [9]:
if not os.path.exists(data_path):
    os.mkdir(data_path)
df.to_csv(f"{data_path}/videos.csv", index=False)